# (TPU) Fine-tuning KB-ALBERT with `Transformers` TensorFlow ver.

Hugging Face의 Transformers는 미세조정(fine-tuning)을 위해 필요한 API들을 간단하게 사용할 수 있는 형태로 제공합니다.
기초적인 파이썬 프로그래밍과 기계학습 모델의 학습 방법만 알고있다면, 자연어처리(NLP)를 전공하지 않은 일반 사용자도 쉽게 최신 인공지능 언어모델을 불러와 사용할 수 있습니다.

그리고 특정 목적을 위해 미세조정된 기계학습 모델을 새로운 데이터에서 쉽게 예측해 볼 수 있도록 Inference Pipeline도 API로 제공하고 있습니다. 그래서 누구나 쉽게 자신이 학습한 모델을 PyTorch나 TensorFlow의 naive programming을 하지 않아도 쉽게 모델을 학습하고 학습된 모델을 배포할 수 있습니다.


<br>

## Objective
- Hugging Face's Transformers를 활용한 fine-tuning
- TPU를 사용하여 fine-tuning

<br>

---

<br>

## 네이버 영화리뷰 감성분석 (Naver Movie Review Sentiment Analysis)

Naver sentiment movie corpus([link](https://github.com/e9t/nsmc))는 한국어 영화 리뷰 데이터로 리뷰 내용의 긍정과 부정이 라벨링 된 데이터입니다. 15만 개의 학습 데이터와 5만 개의 테스트 데이터로 나누어져 있습니다. 140개 이하의 짧은 문장으로 되어 있고, 쉽게 접근하여 사용해볼 수 있는 데이터입니다.

<br>

## Contents

1. 데이터 준비 & 필요 소스코드 다운로드
2. Tokenizer를 활용한 학습용 데이터셋 생성
3. 학습 하이퍼파라미터 설정
4. Head를 활용한 모델 Fine-tuning
5. Inference를 위한 Pipeline 생성

<br>
<br>

# 1. 데이터 및 필요 소스코드 등 준비

예제를 수행하기 위해 필요한 데이터와 소스코드를 다운로드 해야 합니다.
그리고 아쉽게도 KB-ALBERT는 내부 라이선스 이슈로 Transformers Model Hub에 업로드되어 있지 않아 별도의 과정을 통해 요청하신 후에 다운로드 받아 사용할 수 있습니다. 요청 방법은 [링크](https://github.com/KB-BANK-AI/KB-ALBERT-KO/kb-albert-char)를 참고해주세요.

> 참고: 본 예제는 음절단위 모델을 기준으로 작성되었습니다.

<br>

실행 내용
1. Download source code for Custom Tokenizer
2. Install Transformers library
3. Download NSMC dataset

**원활한 모델 학습을 위해 GPU 환경에서 테스트하시기를 권장드립니다.<br> 위 메뉴창에서 "런타임" > "런타임 유형변경" > "하드웨어가속기 <font color='red'>TPU 선택</font>" > "저장" 으로 환경을 변경하실 수 있습니다.**

In [ ]:
# Download source codes
!git clone https://github.com/sackoh/pycon-korea-2020-kb-albert.git
%cd pycon-korea-2020-kb-albert/

# Install transformers
%pip install -q transformers

# Download NSMC dataset
!wget -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

Cloning into 'pycon-korea-2020-kb-albert'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
/content/pycon-korea-2020-kb-albert
     |████████████████████████████████| 890kB 3.4MB/s 
     |████████████████████████████████| 3.0MB 17.0MB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
     |████████████████████████████████| 1.1MB 41.0MB/s 


<br>
<br>

## Upload pretrained model and configuration files

다운로드 받은 파일을 **Google Colab**에서 사용하기 위해서는 파일 업로드가 필요합니다. 아래의 코드를 실행해주세요. 아래를 실행하면 파일 업로드를 위한 팝업창이 열립니다. 다음의 4가지 파일들을 *반드시* 선택하여 업로드를 진행해주세요.

- config.json
- pytorch_model.bin
- tokenizer_config.json
- vocab.txt

업로드에 다소 시간이 걸릴 수 있습니다.

In [ ]:
from google.colab import files
uploaded = files.upload()

%mkdir kb-albert-char
%mv config.json pytorch_model.bin tokenizer_config.json vocab.txt ./kb-albert-char

Saving config.json to config.json
Saving pytorch_model.bin to pytorch_model.bin
Saving tokenizer_config.json to tokenizer_config.json
Saving vocab.txt to vocab.txt


<br>
<br>

## Load train & test data

NSMC 데이터를 불러옵니다. train data와 test data는 각각 리뷰 텍스트와 리뷰의 긍부정 라벨로 구성되어 있습니다.

| text | label |
| ---  | ---   |
| 아 더빙.. 진짜 짜증나네요 목소리 | 0 |
| 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 | 1 |
| 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ... | 1 |

In [ ]:
import csv
from pathlib import Path

def read_nsmc_data(file_path):
    texts = []
    labels = []
    with open(file_path, "r", encoding="utf-8") as r:
        reader = csv.reader(r, delimiter="\t")
        next(reader, None)
        for line in reader:
            texts.append(line[1])
            labels.append(int(line[2]))
    return texts, labels

data_dir = Path('./')
train_texts, train_labels = read_nsmc_data(data_dir/'ratings_train.txt')
test_texts, test_labels = read_nsmc_data(data_dir/'ratings_test.txt')

<br>
<br>

# 2. 학습 하이퍼파라미터 설정

`Transformers`에서 제공하는 `TFTrainingArguments`에 학습 하이퍼파라미터를 설정해줍니다.

설정하지 않은 하이퍼파라미터들은 default 값을 사용합니다.

하이퍼파라미터 설정을 위한 상세 내용은 [링크](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingargument#transformers.TFTrainingArguments)를 참조해주시기 바랍니다.

## GCP Bucket 사용을 위해 GCP project 권한 연동

<font color='#880049'>TPU를 사용하기 위해서는 GCP Storage가 있어야 합니다.
Google Cloud에서 Storage를 생성한 후에 다음의 작업을 진행해야 합니다.
관련된 내용은 [link](https://medium.com/fenwicks/tutorial-0-setting-up-google-colab-tpu-runtime-and-cloud-storage-b88d34aa9dcb) 를 참조해주세요.</font>

In [ ]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project {GCP_PROJECT}

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Updated property [core/project].


<font color='#880049'>TPU를 위해서는 fine-tuning 모델이 생성되는 경로와 log가 생성되는 경로가 GCP bucket 경로이어야 합니다.</font>

In [ ]:
from transformers import TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir = "gs://{GCP_PROJECT}/{GCP_BUCKET}/" #@param {type:"string"}
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir = "gs://{GCP_PROJECT}/{GCP_BUCKET}/logs" #@param {type:"string"}
    logging_steps=2000
)

<br>
<br>

# 3. Tokenizer를 활용한 학습용 데이터셋 생성



## Custom Tokenizer 불러오기

본 예제는 음절단위(Character-level) 모델을 사용하기 때문에 한국어를 위한 음절단위 Tokenizer를 사용해야 합니다. `Transformers`에서 공식 Tokenizer로 등록이 안됐기 때문에 이전 단계에서 다운로드한 소스코드에서 Custom Tokenizer를 불러옵니다.

만약 공식 Hub에 등록된 Bert multilingual 토크나이저를 사용한다면 다음과 같이 불러오면 됩니다.
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
```

In [ ]:
from tokenization_kbalbert import KbAlbertCharTokenizer

tokenizer = KbAlbertCharTokenizer.from_pretrained('kb-albert-char')

# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

<br>
<br>

## Tokenizer를 통해 Raw text를 전처리

불러온 tokenizer를 활용해 input 데이터를 전처리합니다. 텍스트를 sparse indices 형태로 변환하는 것이 주요 처리 내용입니다.

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

<br>
<br>

## Tensorflow Dataset으로 학습용 데이터셋 생성

전처리된 데이터를 텐서플로의의 `Dataset` 안의 `from_tensor_slice`로 만들어줍니다. `from_tensor_slices`와 관련된 자세한 내용은 [링크](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices)를 참조해주세요.

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
   dict(train_encodings),
   train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

<font color='#880049'> TPU는 ditributed 환경을 사용하기 때문에 tensorflow tensor dataset을 그에 맞게 만들어줍니다. </font>

In [ ]:
import math

def get_tfdataset(dataset, args):
    num_examples = tf.data.experimental.cardinality(dataset).numpy()

    if num_examples < 0:
        raise ValueError("The training dataset must have an asserted cardinality")

    approx = math.floor if args.dataloader_drop_last else math.ceil
    steps = approx(num_examples / args.eval_batch_size)
    ds = (
        test_dataset.repeat()
        .batch(args.eval_batch_size, drop_remainder=args.dataloader_drop_last)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    return args.strategy.experimental_distribute_dataset(ds), steps, num_examples


train_ds, _, train_num_examples = get_tfdataset(train_dataset, training_args)
test_ds, test_steps, test_num_examples = get_tfdataset(test_dataset, training_args)

<br>
<br>

# 4. Head를 활용한 모델 Fine-tuning

네이버 영화리뷰 감성분석을 위해 텍스트(sequence of words)에서 긍부정(1/0)을 예측하는 모델을 학습합니다. Sequence의 분류문제(Classification) 미세조정을 위한 Head인 `XXXForSequenceClassification`을 불러옵니다. 본 예제는 ALBERT 언어모델을 사용하고 텐서플로를 활용할 예정이기 때문에 `TFAlbertForSequenceClassification` Head class를 불러옵니다.

## Fine-tuning을 위한 Head에 사전학습한 언어모델 불러오기

Head의 `from_pretrained` 메소드를 통해 사전학습한 KB-ALBERT 언어모델을 함께 불러옵니다. 이때 인자값으로 모델이 위치한 디렉토리(폴더) 경로를 넘겨줍니다.

> Head는 Pretrained Language Model과 Output Layer로 구성되어 있습니다.

미세조정을 통해 Language Model과 Output Layer의 weight가 함께 조정됩니다.

사전학습된 언어모델은 pytorch 모델이기 때문에 `from_pretrained`로 불러올 때 `from_pt=True`로 설정해주어어야 합니다.

BERT multilingual 모델 사용을 위해서는 다음과 같입 입력해주어야 합니다.
```python
from transformers import TFBertForSequenceClassification

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
```

In [ ]:
from transformers import TFAlbertForSequenceClassification

with training_args.strategy.scope():
    model = TFAlbertForSequenceClassification.from_pretrained('kb-albert-char', from_pt=True)


# from transformers import TFBertForSequenceClassification
# with training_args.strategy.scope():
#     model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFAlbertForSequenceClassification from a TF 2.0 model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a TFBertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from a TF 2.0 model that you expect to be exactly identical (e.g. initializing a BertForSequenceClassification model from a TFBertForSequenceClassification model).
Some weights or buffers of the PyTorch model TFAlbertForSequenceClassification were not initialized from the TF 2.0 model and are newly initialized: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dens

<br>
<br>

## Trainer를 통해 fine-tuning 수행

그 다음은 학습을 위해 `Trainer` class를 불러옵니다. Head와 학습 하이퍼파라미터 그리고 앞에서 생성한 학습용 데이터셋을 인자로 넘긴 후에 `train()` 메소드로 학습을 시작합니다.


In [ ]:
#@title Customized Trainer for TPU (Shift + Enter)
import os
import datetime
import tensorflow as tf
from transformers import TFTrainer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR, EvalPrediction, PredictionOutput, set_seed
from transformers.utils import logging
from tqdm.auto import tqdm, trange


logger = logging.get_logger(__name__)


class TFTrainer_(TFTrainer):

    def train(self) -> None:
        """
        Train method to train the model.
        """
        train_ds = self.train_dataset
        self.total_train_batch_size = self.args.train_batch_size * self.args.gradient_accumulation_steps
        self.num_train_examples = 150000

        if self.args.debug:
            tf.summary.trace_on(graph=True, profiler=True)

        self.gradient_accumulator.reset()

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.steps_per_epoch = self.args.max_steps
        else:
            approx = math.floor if self.args.dataloader_drop_last else math.ceil
            self.steps_per_epoch = approx(self.num_train_examples / self.total_train_batch_size)
            t_total = self.steps_per_epoch * self.args.num_train_epochs

        with self.args.strategy.scope():
            self.create_optimizer_and_scheduler(num_training_steps=t_total)
            iterations = self.optimizer.iterations
            self.global_step = iterations.numpy()
            folder = os.path.join(self.args.output_dir, PREFIX_CHECKPOINT_DIR)
            ckpt = tf.train.Checkpoint(optimizer=self.optimizer, model=self.model)
            self.model.ckpt_manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=self.args.save_total_limit)

            if self.model.ckpt_manager.latest_checkpoint:
                epochs_trained = self.global_step // (self.num_train_examples // self.args.gradient_accumulation_steps)
                steps_trained_in_current_epoch = self.global_step % (
                    self.num_train_examples // self.args.gradient_accumulation_steps
                )

                logger.info("  Continuing training from checkpoint, will skip to saved global_step")
                logger.info("  Continuing training from epoch %d", epochs_trained)
                logger.info("  Continuing training from global step %d", self.global_step)
                logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
                logger.info(
                    "Checkpoint file %s found and restoring from checkpoint", self.model.ckpt_manager.latest_checkpoint
                )

                ckpt.restore(self.model.ckpt_manager.latest_checkpoint).expect_partial()
            else:
                epochs_trained = 1

            tf.summary.experimental.set_step(iterations)

            epochs = 1 if self.args.max_steps > 0 else self.args.num_train_epochs

            if self.args.fp16:
                policy = tf.keras.mixed_precision.experimental.Policy("mixed_float16")
                tf.keras.mixed_precision.experimental.set_policy(policy)

            with self.tb_writer.as_default():
                tf.summary.text("args", self.args.to_json_string())

            self.tb_writer.flush()

            logger.info("***** Running training *****")
            logger.info("  Num examples = %d", self.num_train_examples)
            logger.info("  Num Epochs = %d", epochs)
            logger.info("  Instantaneous batch size per device = %d", self.args.per_device_train_batch_size)
            logger.info(
                "  Total train batch size (w. parallel, distributed & accumulation) = %d", self.total_train_batch_size
            )
            logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
            logger.info("  Steps per epoch = %d", self.steps_per_epoch)
            logger.info("  Total optimization steps = %d", t_total)

            self.train_loss = tf.keras.metrics.Sum()
            start_time = datetime.datetime.now()

            train_iterator = tqdm(range(epochs_trained, int(epochs + 1)), desc="Epoch")
            for epoch_iter in train_iterator:
                # Reset the past mems state at the beginning of each epoch if necessary.
                if self.args.past_index >= 0:
                    self._past = None

                epoch_iterator = tqdm(train_ds, desc="Iteration")
                for step, batch in enumerate(epoch_iterator):
                    self.global_step = iterations.numpy()
                    self.epoch_logging = epoch_iter - 1 + (step + 1) / self.steps_per_epoch

                    self.distributed_training_steps(batch)

                    training_loss = self.train_loss.result() / ((step + 1) * self.total_train_batch_size)

                    if self.args.debug:
                        logs = {}
                        logs["loss"] = training_loss.numpy()
                        logs["epoch"] = self.epoch_logging

                        self.log(logs)

                    if self.global_step == 1 and self.args.debug:
                        with self.tb_writer.as_default():
                            tf.summary.trace_export(
                                name="training", step=self.global_step, profiler_outdir=self.args.logging_dir
                            )

                    if (
                        self.global_step > 0
                        and self.args.evaluate_during_training
                        and self.global_step % self.args.eval_steps == 0
                    ):
                        self.evaluate()

                    if (self.global_step > 0 and self.global_step % self.args.logging_steps == 0) or (
                        self.global_step == 1 and self.args.logging_first_step
                    ):
                        logs = {}
                        logs["loss"] = training_loss.numpy()
                        logs["learning_rate"] = self.lr_scheduler(self.global_step).numpy()
                        logs["epoch"] = self.epoch_logging

                        self.log(logs)

                    if self.global_step > 0 and self.global_step % self.args.save_steps == 0:
                        ckpt_save_path = self.model.ckpt_manager.save()

                        logger.info("Saving checkpoint for step {} at {}".format(self.global_step, ckpt_save_path))

                    if self.global_step > 0 and self.global_step % self.steps_per_epoch == 0:
                        break

                self.train_loss.reset_states()

            end_time = datetime.datetime.now()

            logger.info("Training took: {}".format(str(end_time - start_time)))

        if self.args.past_index and hasattr(self, "_past"):
            # Clean the state at the end of training
            delattr(self, "_past")

    def evaluate(self, eval_ds, steps, num_examples):
        """
        Run evaluation and returns metrics.
        The calling script will be responsible for providing a method to compute metrics, as they are
        task-dependent (pass it to the init :obj:`compute_metrics` argument).
        Args:
            eval_dataset (:class:`~tf.data.Dataset`, `optional`):
                Pass a dataset if you wish to override :obj:`self.eval_dataset`.
        Returns:
            A dictionary containing the evaluation loss and the potential metrics computed from the predictions.
        """
        print("Evaluating...")

        with self.args.strategy.scope():
            output = self.prediction_loop(eval_ds, steps, num_examples, description="Evaluation")
            logs = {**output.metrics}
            logs["epoch"] = self.epoch_logging

            self.log(logs)
            print(output.metrics)

        return output.metrics

    @tf.function
    def distributed_prediction_steps(self, batch):
        with self.args.strategy.scope():
            logits = self.args.strategy.run(self.prediction_step, batch)

        return logits


In [ ]:
trainer = TFTrainer_(
    model=model,
    args=training_args,
    train_dataset=train_ds,
)

trainer.train()

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


In [ ]:
trainer = TFTrainer_(
    model=model,
    args=training_args,
    train_dataset=train_ds,
)


<br>
<br>

# 5. Inference를 위한 Pipeline 생성

`Transformers`는 미세조정된 모델에 raw text를 인풋으로 넣었을 때 데이터 전처리와 모델 추론 과정을 `pipeline` class를 통해 api 형태로 쉽게 개발할 수 있도록 하였습니다.

pipeline에는 사전 정의된 task 유형과 미세조정된 모델, tokenizer 그리고 딥러닝 프레임워크 유형을 인자로 전달합니다. (PyTorch는 'pt', TensorFlow는 'tf')

생성된 pipeline 인스턴스에 raw text를 넣어주면 예측 결과와 예측 결과에 대한 confidence 값이 반환됩니다.

In [ ]:
from transformers import pipeline

nsmc_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, framework='tf')
id2label = {"LABEL_0": "negative", "LABEL_1": "positive"}

reviews= [
          "생각한 것보다는 영화가 재미없었어",
          "OO!",
          "역시 재밌네ㅋㅋ 최고"
]

results = nsmc_classifier(reviews)
for idx, result in enumerate(results):
    print(reviews[idx])
    for k, v in result.items():
        print(f" >> {k} : {id2label[v] if k == 'label' else v}")
    print()

생각한 것보다는 영화가 재미없었어
 >> label : negative
 >> score : 0.998558521270752

OO!
 >> label : negative
 >> score : 0.9973821640014648

역시 재밌네ㅋㅋ 최고
 >> label : positive
 >> score : 0.9665402770042419



pipeline이나 미세조정된 모델은 `save_pretrained`를 통해 지정된 경로에 저장할 수 있습니다. 다시 사용해야 할 때는 같은 경로명을 `from_pretrained`에 넣어주어 쉽게 불러올 수 있습니다.

In [ ]:
nsmc_classifier.save_pretrained('./pipeline')